In [ ]:
import sounddevice as sd
import soundfile as sf
import numpy as np
from IPython.display import Audio 

# Define the sample rate and duration of the recording
sample_rate = 16000  # in Hz
duration = 5  # in seconds

# Record audio
audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
sd.wait()  # Wait for the recording to finish

# Process the audio into the format expected by your model
# This step will depend on your specific model
audio = np.squeeze(audio)

# Save the audio to a .wav file
sf.write('recording.wav', audio, sample_rate)

Audio('recording.wav')

In [ ]:
import deepspeech
import wave
import numpy as np

def transcribe_audio(file_path, model_path):
    # Load DeepSpeech model
    model = deepspeech.Model(model_path)

    # Read audio file
    with wave.open(audio_file_path, 'rb') as audio_file:
        audio_data = audio_file.readframes(audio_file.getnframes())
        audio_data = np.frombuffer(audio_data, dtype=np.int16)

    # Perform speech-to-text
    text = model.stt(audio_data)

    print("Transcript: {}".format(text))

# Example usage:
audio_file_path = 'recording.wav'
model_path = 'Model Arabic Transcription.pbmm'

transcribe_audio(audio_file_path, model_path)


Transcript: أليللر معن


In [ ]:
from pytube import YouTube
from pydub import AudioSegment
import wave
import numpy as np
import deepspeech

def download_youtube_video(youtube_url, output_path):
    yt = YouTube(youtube_url)
    yt.streams.filter(only_audio=True).first().download(output_path)

def convert_audio_format(input_path, output_path, target_sample_rate=16000, target_channels=1, target_sample_width=2):
    audio = AudioSegment.from_file(input_path)
    
    # Resample if needed
    if audio.frame_rate != target_sample_rate:
        audio = audio.set_frame_rate(target_sample_rate)
    
    # Convert to mono if needed
    if audio.channels > 1 and target_channels == 1:
        audio = audio.set_channels(1)
    
    # Save as WAV
    audio.export(output_path, format="wav")

def transcribe_audio(file_path, model_path):
    model = deepspeech.Model(model_path)
    
    with wave.open(file_path, 'rb') as audio_file:
        audio_data = audio_file.readframes(audio_file.getnframes())
        audio_data = np.frombuffer(audio_data, dtype=np.int16)
    
    text = model.stt(audio_data)
    
    print("Transcript: {}".format(text))

# Example usage:
youtube_url = "https://www.youtube.com/watch?v=xv1TQvROLfk"
output_audio_path = "Surah Nas.mp3"  # Using mp3 format for initial download
converted_audio_path = "Surah Nas.wav"
model_path = "Model Arabic Transcription.pbmm"

# Download YouTube video
#download_youtube_video(youtube_url, output_audio_path)

# Convert audio format
#convert_audio_format(output_audio_path, converted_audio_path)

# Transcribe audio using DeepSpeech
transcribe_audio(converted_audio_path, model_path)


Transcript: عععععععععععععععععععععععععععععععععععععععععععععاااعععععععحععععععععععععععععععععععععععععععععع


This is a hugging face transformer!

In [1]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

# Now yo


CUDA is available. Using GPU.


In [2]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import soundfile as sf

LANG_ID = "ar"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-arabic"

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

# Specify the path to your .wav file
wav_file_path = "Surah Nas.wav"

# Load the audio file
audio_data, _ = sf.read(wav_file_path, dtype='int16')

# Ensure the audio data is 1D (remove extra dimensions)
audio_data = torch.tensor(audio_data.flatten(), dtype=torch.float32).unsqueeze(0)

# Move the model and audio data to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
audio_data = audio_data.to(device)

# Perform inference
with torch.no_grad():
    logits = model(audio_data).logits

# Decode the logits to text (replace with actual decoding logic)
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]

# Print the transcription
print("Transcription:", transcription)


c:\Users\Ahmad\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-arabic were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exact

OutOfMemoryError: CUDA out of memory. Tried to allocate 14.71 GiB. GPU 0 has a total capacty of 4.00 GiB of which 73.50 MiB is free. Of the allocated memory 1.65 GiB is allocated by PyTorch, and 1.48 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [7]:
import torch
import sounddevice as sd
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

LANG_ID = "ar"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-arabic"

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

# Real-time audio streaming function
def audio_callback(indata, frames, time, status):
    if status:
        print(status, flush=True)

    # Ensure the audio data is 1D (remove extra dimensions)
    audio_data = torch.tensor(indata.flatten(), dtype=torch.float32).unsqueeze(0)

    # Perform real-time inference
    with torch.no_grad():
        logits = model(audio_data).logits

    # Decode the logits to text (replace with actual decoding logic)
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    # Print the transcription
    print("Transcription:", transcription, flush=True)

# Set up the audio stream
block_size = 2048
samplerate = 44100  # Adjust the samplerate according to your needs
with sd.InputStream(callback=audio_callback, channels=1, blocksize=1024, samplerate=samplerate, dtype="int16"):
    print("Listening for real-time audio. Press Ctrl+C to exit.")
    input("Press Enter to stop the audio stream.")


Listening for real-time audio. Press Ctrl+C to exit.
Transcription: 
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
Transcription: 
input overflow
T